In [1]:
!pip install transforms

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for transforms: filename=transforms-0.1-py3-none-any.whl size=39351 sha256=d46945de907e0916ce847958e26e5090045c0734a39b3e8228709cbe426a5071
  Stored in directory: /root/.cache/pip/wheels/73/8d/26/be8b7e278f01d5093d18fc99c8158bd8528403242c852a558c
Successfully built transforms


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 29.5 MB/s 
     |████████████████████████████████| 6.6 MB 48.8 MB/s 
     |████████████████████████████████| 163 kB 69.5 MB/s 


In [3]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 431 kB 32.0 MB/s 
     |████████████████████████████████| 115 kB 72.8 MB/s 
     |████████████████████████████████| 212 kB 52.6 MB/s 
     |████████████████████████████████| 127 kB 63.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [4]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.6 MB/s 


In [5]:
import numpy as np
import pandas as pd

from datasets import load_dataset, load_metric
from transformers import AutoTokenizer

In [6]:
corpus = load_dataset('merionum/ru_paraphraser')

/usr/local/lib/python3.7/dist-packages/huggingface_hub/utils/_deprecation.py:97: FutureWarning: Deprecated argument(s) used in 'dataset_info': token. Will not be supported from version '0.12'.
  warnings.warn(message, FutureWarning)


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-aad01005423eb70c/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
batch_size = 16

In [8]:
model_name = "IlyaGusev/xlm_roberta_large_headline_cause_simple"
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/423 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [9]:
corpus

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
        num_rows: 1924
    })
})

In [10]:
label_list = sorted(set(corpus['train']['class']))
label_list

['-1', '0', '1']

In [11]:
labels2id = { key:id for id, key in enumerate(label_list)}
labels2id

{'-1': 0, '0': 1, '1': 2}

In [12]:
id2labels = { id:key for id, key in enumerate(label_list)}
id2labels

{0: '-1', 1: '0', 2: '1'}

In [13]:
def tokenize_and_align_labels(tokenizer, labels2id):    
    def tokenize_and_align_labels_(examples):
        tokenized_inputs = tokenizer(examples['text_1'],examples['text_2'], truncation=True)
        tokenized_inputs["labels"] = [labels2id[label] for label in examples['class']]
        return tokenized_inputs
    return tokenize_and_align_labels_

In [14]:
tokenized_input = tokenize_and_align_labels(tokenizer, labels2id)(corpus['train'][20:24])
tokenized_input

{'input_ids': [[0, 18449, 1068, 1951, 169066, 59, 42136, 59392, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2, 2, 6, 144048, 145158, 184643, 18449, 28196, 529, 68056, 49869, 159288, 6, 32345, 14539, 25455, 5, 2], [0, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 7329, 155333, 24863, 5, 2, 2, 417, 33372, 526, 70847, 132653, 326, 175857, 529, 861, 29034, 534, 11591, 149673, 5, 2], [0, 417, 14100, 695, 1200, 49, 30037, 1269, 326, 3699, 108854, 1857, 56869, 90229, 546, 209, 9561, 5, 2, 2, 417, 68642, 69, 53177, 29, 108854, 1857, 47745, 49, 14100, 695, 1200, 90229, 546, 209, 9561, 5, 2], [0, 72085, 138109, 87830, 29, 23262, 2294, 90569, 11974, 24744, 59, 161679, 8568, 5, 2, 2, 69242, 64393, 41976, 4560, 1560, 182076, 29, 29230, 23262, 49, 72085, 138109, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [15]:
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"][0])
print(tokens)

['<s>', '▁Че', 'ч', 'ня', '▁попросил', 'а', '▁национал', 'истов', '▁со', 'ставить', '▁кодекс', '▁поведения', '▁', 'кав', 'каз', 'цев', '.', '</s>', '</s>', '▁', 'Национал', 'исты', '▁помогут', '▁Че', 'чне', '▁со', 'ставить', '▁кодекс', '▁поведения', '▁', 'кав', 'каз', 'цев', '.', '</s>']


In [16]:
corpus['train'][100]

{'id': '103',
 'id_1': '112',
 'id_2': '4234',
 'text_1': 'Renault сократит 7,5 тыс. сотрудников из-за кризиса.',
 'text_2': 'Renault сократит 7,5 тысячи рабочих мест во Франции.',
 'class': '1'}

In [17]:
tokenized_datasets = corpus.map(tokenize_and_align_labels(tokenizer, labels2id), batched=True)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 7227
    })
    test: Dataset({
        features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1924
    })
})

In [19]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [20]:
from transformers import AutoModelForSequenceClassification, BertForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=len(label_list))

model.config.id2label = id2labels 
model.config.label2id = labels2id

Downloading:   0%|          | 0.00/945 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [21]:
from datasets import load_metric

metric = load_metric("accuracy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)
    
    return metric.compute(predictions=predictions, references=labels)

In [23]:
args = TrainingArguments(
    "paraphras",
    evaluation_strategy = "epoch",
    learning_rate=2e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.05,
    save_strategy='no',
    report_to='none',
)

In [24]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [25]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_1, id_2, text_1, class, id. If text_2, id_1, id_2, text_1, class, id are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 7227
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2260
You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.728547,0.656965
2,0.756600,0.700382,0.692827
3,0.588300,0.686240,0.705301
4,0.537800,0.694904,0.713617
5,0.498800,0.729213,0.709459


The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_1, id_2, text_1, class, id. If text_2, id_1, id_2, text_1, class, id are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1924
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_1, id_2, text_1, class, id. If text_2, id_1, id_2, text_1, class, id are not expected by `XLMRobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1924
  Batch size = 16
The following columns in the evaluation set don't have a corresponding argument in `XLMRobertaForSequenceClassification.forward` and have been ignored: text_2, id_1, id_2, text_1, class,

TrainOutput(global_step=2260, training_loss=0.583625192557816, metrics={'train_runtime': 1679.4071, 'train_samples_per_second': 21.517, 'train_steps_per_second': 1.346, 'total_flos': 3034369670565798.0, 'train_loss': 0.583625192557816, 'epoch': 5.0})

In [26]:
example = tokenized_datasets["test"][40:80]

In [27]:
import torch 

tokens = tokenizer(example['text_1'], example['text_1'], padding=True, truncation=True, return_tensors='pt')
tokens = tokens.to('cuda:0')

with torch.no_grad():
    outputs = model(**tokens)


predicted = outputs.logits.argmax(dim=-1).cpu().numpy()
classes = [id2labels[id_label] for id_label in predicted]

In [28]:
df_example =pd.DataFrame({'text_1':example['text_1'], 'text_2':example['text_2'], 'class':example['class'], 'predict':classes})
df_example

,text_1,text_2,class,predict
0,Владимир Путин освободил от должности почти 20...,Путин снял с должностей более 20 руководителей...,0,1
1,Источник: Якунин озвучил свою месячную зарплат...,Якунин назвал размер своей зарплаты,0,1
2,Apple получила патент на социальную сеть,Apple получила патент на собственную соцсеть,1,1
3,Подвесной потолок обрушился в аэропорту Стамбу...,Четверо пострадали при обрушении подвесного по...,1,1
4,Россия представляет принципиально новый танк,Вашингтон пригрозил России новыми санкциями,-1,1
5,Генсек ООН и премьер Украины обсудили минские ...,Генсек ООН и премьер-министр Украины обсудили ...,1,1
6,Результаты выборов в Великобритании укрепили фунт,Стали известны окончательные результаты выборо...,-1,1
7,Николич: Сербия не говорила о нежелании строит...,Президент Сербии опроверг свои слова об отказе...,0,1
8,СМИ сообщают об обысках в Минкульте по делу о ...,Минкультуры отчиталось перед следствием по дел...,-1,1
9,Любляна отпразднует День Победы вместе с Москвой,Путеводитель по Дню Победы: как провести 9 мая...,-1,1
